In [149]:
import praw

In [150]:
reddit = praw.Reddit(client_id='redacted',
                    client_secret= 'redacted',
                    user_agent='redacted',
                    username='redacted',
                    password='redacted',)

In [151]:
#Poll Reddit by sub

subs = [{'subName': 'LiverpoolFC', 'postTitle': 'FT', 'postFlair': 'Post Match'},
        {'subName': 'Gunners', 'postTitle': 'Post Match', 'postFlair': 'Post-Match Thread'},
        {'subName': 'RedDevils', 'postTitle': 'Post Match', 'postFlair': None},
        {'subName': 'MCFC', 'postTitle': 'Post Match', 'postFlair': 'Match Day'},
        {'subName': 'coys', 'postTitle': 'Thread', 'postFlair', 'PostMatch Thread'},
        {'subName': 'chelseafc', 'postTitle', 'Thread', 'postFlair', 'Post-Match Thread'}
       ]

sub='LiverpoolFC'
subReddit = reddit.subreddit(sub)


SyntaxError: invalid syntax (739012568.py, line 7)

In [152]:
end_of_season_2223 = 1685418710
beginning_of_season2223 = 1659744000
print(subs[0])

{'subName': 'LiverpoolFC', 'postTitle': 'Post Match', 'postFlair': 'Post Match'}


In [153]:
submission_list_liverpool = [
    submission for submission in reddit.subreddit('LiverpoolFC').search('FT') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post Match'
]
submission_list_arsenal = [
    submission for submission in reddit.subreddit('Gunners').search('Post Match') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post-Match Thread'

]

submission_list_manunited = [
    submission for submission in reddit.subreddit('RedDevils').search('Post Match') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223

]

submission_list_mancity = [
    submission for submission in reddit.subreddit('MCFC').search('Post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Match Day'

]

submission_list_spurs = [
    submission for submission in reddit.subreddit('coys').search('post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'PostMatch Thread'

]

submission_list_chelsea = [
    submission for submission in reddit.subreddit('chelseafc').search('post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post-Match Thread'

]

In [154]:
import pandas as pd

def get_comment_data(end, beginning, submission_list):
    data = pd.DataFrame()

    for i in submission_list:
        submission = reddit.submission(i)
        finaltitle = ''
        title = i.title.split()
        for i in range(len(title)):
            if ']' in title[i]:
                newtitle = title[i+1::]
                finaltitle = ' '.join(newtitle)
        data[finaltitle] = 0

        comment_data = []

        submission.comments.replace_more(limit=None)

        for comment in submission.comments.list():
            comment_data.append(str(comment.body))


        ser = pd.Series(comment_data)
        data[finaltitle] = ser
    return data


In [156]:
# data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_liverpool)

In [ ]:
# data2223

In [23]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [44]:
def get_mood_ratio(data):

    moodDF = pd.DataFrame()
    #for each column in the data
    for col in data.columns:
        col_list = []
        #for each comment in each column
        for comment in data[col].dropna():
            #split text into words
            temp = comment.split(' ')
            #find length
            length = len(temp)
            #need to restrict length of paragraph so that the sentiment pipeline can work
            if length > 375:
                new_comment = ' '.join(temp[:375])
                sent = sentiment_pipeline(new_comment)
                col_list.append(sent[0])

            else:
                sent = sentiment_pipeline(comment)
                col_list.append(sent[0])
        
        #
        moodDF.loc[:, col] = pd.Series(col_list)
        
        
    #cleaning moodDF, getting games out if there arent many comments
    new_moodDF = moodDF.copy()
    for column in new_moodDF.columns: 
        if new_moodDF[column].count() <= 7: 
            new_moodDF=new_moodDF.drop(column, axis=1) 
    
    #now making percentage
    percentage = pd.DataFrame()

    for col in new_moodDF.columns:
        num = 0
        num_negative = 0
        for item in new_moodDF[col].dropna():
            if item['label'] == 'NEGATIVE':
                num_negative += 1
            num += 1
        percentage[col] = [(num_negative/num)]
    
    return percentage
moodDF = get_mood_ratio(data)

In [45]:
moodDF

,LIVERPOOL FC 2 - 5 REAL MADRID,CRYSTAL PALACE 0 - 0 LIVERPOOL FC,LIVERPOOL FC 2 - 0 WOLVES,WOLVES 3 - 0 LIVERPOOL FC,NEWCASTLE 0 - 2 LIVERPOOL FC,LIVERPOOL FC 2 - 0 EVERTON,BRIGHTON 2 - 1 LIVERPOOL FC,,BRIGHTON 3 - 0 LIVERPOOL FC,WOLVES 0 - 1 LIVERPOOL FC & LIVERPOOL FC 0 - 0 CHELSEA,...,Rangers 1 - 7 Liverpool,Arsenal 3 - 2 Liverpool,Everton 0 - 0 Liverpool,Liverpool 3 - 3 Brighton,LIVERPOOL FC 1 - 1 CRYSTAL PALACE,Liverpool 9 - 0 Bournemouth,Manchester United 2 - 1 Liverpool,Man United 2 - 1 Liverpool,Napoli 4 - 1 Liverpool,Liverpool 1 - 1 Crystal Palace
0,0.73913,0.685393,0.6,0.626866,0.357143,0.423077,0.714286,0.702381,0.653846,0.727273,...,0.384615,0.47619,0.627451,0.727273,0.612245,0.363636,0.658537,0.6875,0.461538,0.538462
